In [1]:
from surface import grammar
from surface import converter
from collections import defaultdict

In [2]:
def make_default_structure(graph_data, word_id):
    if word_id not in graph_data:
        graph_data[word_id] = {
            "word": "",
            "deps": {},
        }

In [3]:
def extract_rules(dev):
    graph_data = {}
    noun_list = []
    id_to_rules = defaultdict(list)
    id_to_sentence = {}
    sentences = 0
    with open(dev, "r") as f:
        for i,line in enumerate(f):            
            if line == "\n":
                words = []
                for w in graph_data:
                    words.append(graph_data[w]["word"])
                    subgraphs = {"root": None, "graph": []}
                    rules = []
                    if "tree_pos" not in graph_data[w]:
                        continue
                    
                    subgraphs["root"] = graph_data[w]["tree_pos"]
                    
                    for dep in graph_data[w]["deps"]:                        
                        edge_dep = graph_data[w]["deps"][dep]
                        to_pos = graph_data[dep]["tree_pos"]
                        mor = graph_data[dep]["mor"]
                            
                        if "tree_pos" in graph_data[w]:
                            if "lin=+" in mor:
                                subgraphs["graph"].append({"to":to_pos, "edge":edge_dep, "dir":"S"})
                            elif "lin=-" in mor:
                                subgraphs["graph"].append({"to":to_pos, "edge":edge_dep, "dir":"B"})
                            else:
                                subgraphs["graph"].append({"to":to_pos, "edge":edge_dep, "dir":None})

                            #if line_key_before not in grammar and line_key_after not in grammar:
                            #    print(line_key_before)

                    id_to_rules[sentences].append(subgraphs)
                    id_to_sentence[sentences] = words
                graph_data = {}
                noun_list = []
                sentences += 1
                continue
            if line.startswith("#"):
                continue
            if line != "\n":
                fields = line.split("\t")
                word_id = fields[0]
                word = fields[1]
                tree_pos = fields[3]
                mor = fields[5]
                head = fields[6]
                ud_edge = fields[7]

                make_default_structure(graph_data, word_id)
                graph_data[word_id]["word"] = word
                graph_data[word_id]["tree_pos"] = tree_pos
                graph_data[word_id]["mor"] = mor

                make_default_structure(graph_data, head)
                graph_data[head]["deps"][word_id] = ud_edge
    return id_to_rules, id_to_sentence

In [4]:
GRAMMAR_FILE = "../count_en_ewt-ud-train.conllu"
TERMINAL_FILE = "../en_ewt-ud-dev.conllu"

In [52]:
rules, sens = extract_rules(TERMINAL_FILE)
sens[1163]

['Buy', '', 'a', 'phone', 'new']

In [65]:
%%capture cap --no-stderr
grammar.generate_grammar(GRAMMAR_FILE, rules[1163])
grammar.generate_terminals(TERMINAL_FILE)

with open('dep_grammar_spec.irtg', 'w') as f:
    f.write(cap.stdout)

In [58]:
converter.convert(TERMINAL_FILE)

In [ ]:
!java -Xmx8G -cp alto-2.3.1-all.jar de.up.ling.irtg.script.ParsingEvaluator graphs_100 -g dep_grammar_spec.irtg -I ud -O string=toString -o surface_eval_ewt